In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [3]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [4]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [5]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [6]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [7]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 1.716 seconds.
Prefix dict has been built succesfully.


In [8]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [9]:
from random import shuffle
word_data[8]

['北山',
 '狼',
 '睡袋',
 '户外',
 '成人',
 '睡袋',
 '信封',
 '式',
 '抓绒',
 '睡袋',
 '内胆',
 '春夏季',
 '睡袋',
 '旅行',
 '隔脏',
 '睡袋',
 '可',
 '拼接',
 '午休',
 '睡袋',
 '蓝']

## 标签分离(一个标签分成三个标签)

In [10]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [11]:
len(data_y["label1"].unique())

22

In [12]:
len(data_y["label2"].unique())

191

In [13]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [14]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [15]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

In [23]:
from keras.preprocessing.text import *
from keras.preprocessing import sequence
tok = Tokenizer()
tok.fit_on_texts(word_data)

In [24]:
tok.word_index

{'新款': 1,
 '1': 2,
 '汽车': 3,
 '时尚': 4,
 '黑色': 5,
 '鞋': 6,
 '2016': 7,
 '休闲': 8,
 '款': 9,
 '专用': 10,
 '2': 11,
 '5': 12,
 '男士': 13,
 '跟': 14,
 '手机': 15,
 '适用': 16,
 '女': 17,
 '套装': 18,
 '男': 19,
 '韩版': 20,
 '夏季': 21,
 '儿童': 22,
 '3': 23,
 '包': 24,
 '于': 25,
 '透气': 26,
 '壳': 27,
 '套': 28,
 '休闲鞋': 29,
 '米': 30,
 '户外': 31,
 '男鞋': 32,
 '真皮': 33,
 '垫': 34,
 '红色': 35,
 '6': 36,
 '4': 37,
 '脚垫': 38,
 '女鞋': 39,
 '男女': 40,
 '白色': 41,
 '全': 42,
 '蓝色': 43,
 '创意': 44,
 '宝宝': 45,
 '防水': 46,
 '简约': 47,
 '四季': 48,
 '女士': 49,
 '8': 50,
 '保护套': 51,
 '凉鞋': 52,
 '牛皮': 53,
 '四件套': 54,
 '皮鞋': 55,
 '系列': 56,
 '专车': 57,
 '情侣': 58,
 '包围': 59,
 '灯': 60,
 '通用': 61,
 '纯棉': 62,
 '加厚': 63,
 '小': 64,
 '运动': 65,
 '单鞋': 66,
 '商务': 67,
 '学生': 68,
 '大': 69,
 '2015': 70,
 '送': 71,
 '潮流': 72,
 '客厅': 73,
 '婴儿': 74,
 '带': 75,
 '新': 76,
 '卡通': 77,
 '号': 78,
 '全棉': 79,
 '英伦': 80,
 '7': 81,
 '0': 82,
 '寸': 83,
 '可': 84,
 '现代': 85,
 '鞋子': 86,
 'led': 87,
 '单肩': 88,
 '玩具': 89,
 '黑': 90,
 '被': 91,
 '座垫': 92,
 '39': 93,
 '礼物': 94,

In [34]:
len(tok.word_index)

274420

In [25]:
x_train = tok.texts_to_sequences(word_data_train)

In [26]:
x_test = tok.texts_to_sequences(word_data_test) 

In [27]:
x_train = sequence.pad_sequences(x_train, maxlen=18)
x_test  = sequence.pad_sequences(x_test, maxlen=18) 

In [28]:
x_test

array([[    0,     0,     0, ...,   882,  2454, 14626],
       [ 6472,    14,    26, ...,   226,   152,   102],
       [    0,     0,     0, ..., 45092,  2574,   133],
       ...,
       [    0,     0,     0, ...,   162, 26206,  1540],
       [    0, 19138,    51, ...,    12,    12,   187],
       [  150,   208,   444, ...,    11,    23,    30]])

# 词向量训练

In [19]:
from gensim.models.word2vec import Word2Vec 
if os.path.exists("w2v_model"):
    model = Word2Vec.load("w2v_model")
    print(1)
else:
    sentences = word_data
    model= Word2Vec(size=50, window=10, min_count = 1)
    model.build_vocab(sentences)
    model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)
    model.save("w2v_model")

1


## 填充词向量(构造输入输出)（让每个句子拥有同样的维度）

In [20]:

x_train = [model[word] for word in word_data_train]
x_test = [model[word] for word in word_data_test]
x_train = sequence.pad_sequences(x_train, maxlen=18)
x_test  = sequence.pad_sequences(x_test, maxlen=18)

c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## 默认参数


## TextCnn

In [88]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, maxlen, sequence_length, num_classes, filter_sizes, embedding_size, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
#         self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        with tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([274420, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
        l2_loss = tf.constant(0.0)
        keep_prob = 0.9
        pooled_outputs = []
        
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                # 卷积
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
        
        
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
                # 最大池化
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled = tf.nn.dropout(pooled, keep_prob)
                pooled_outputs.append(pooled)
                

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        print(self.h_pool.shape)
        
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        self.h_drop = self.h_pool_flat
        
        # Final (unnormalized) scores and predictions
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)

In [89]:
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

In [92]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
maxlen = 16

batch_size = 256 

epochs = 100

is_training = True
# 词向量长度
embedding_dims = 50

filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100

# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
#           allow_soft_placement=FLAGS.allow_soft_placement,
#           log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(maxlen = 10, sequence_length = 18, num_classes = [22, 191, 1192], filter_sizes = [1,2, 3, 4, 5], embedding_size = 32, num_filters = 64)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3
                }
#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), 640, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev
                    }
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [93]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)


(?, 1, 1, 320)
2019-02-23T23:19:11.158880: step 40, loss 4.08853, acc [0.40341797 0.33457031 0.24404297 0.18828125]
2019-02-23T23:19:13.406257: step 80, loss 2.95948, acc [0.60078125 0.503125   0.41552734 0.34140625]
2019-02-23T23:19:15.625855: step 120, loss 2.44486, acc [0.68896484 0.58642578 0.50292969 0.41601562]
2019-02-23T23:19:17.855871: step 160, loss 2.12075, acc [0.73525391 0.63769531 0.55419922 0.47460938]
2019-02-23T23:19:20.048686: step 200, loss 1.89136, acc [0.76757812 0.675      0.58779297 0.51259766]

Evaluation:
2019-02-23T23:19:25.726393: step 200, loss 1.78377, acc [0.78606253 0.68534073 0.60996706 0.53240096] 

2019-02-23T23:19:28.112598: step 240, loss 1.73932, acc [0.79199219 0.69521484 0.61630859 0.53720703]
2019-02-23T23:19:30.360965: step 280, loss 1.62361, acc [0.81289062 0.71347656 0.63496094 0.56484375]
2019-02-23T23:19:32.712004: step 320, loss 1.58074, acc [0.81630859 0.71904297 0.63925781 0.56826172]
2019-02-23T23:19:34.921188: step 360, loss 1.44661, ac

2019-02-23T23:22:17.800246: step 2640, loss 0.658192, acc [0.91533203 0.86738281 0.82041016 0.76308594]
2019-02-23T23:22:19.793173: step 2680, loss 0.631035, acc [0.92050781 0.87441406 0.82373047 0.765625  ]
2019-02-23T23:22:21.787094: step 2720, loss 0.636812, acc [0.91992188 0.8765625  0.82460937 0.77099609]
2019-02-23T23:22:23.784485: step 2760, loss 0.646809, acc [0.91689453 0.86923828 0.82167969 0.76396484]
2019-02-23T23:22:25.776915: step 2800, loss 0.644426, acc [0.91357422 0.86914062 0.82744141 0.77207031]

Evaluation:
2019-02-23T23:22:29.205873: step 2800, loss 0.513215, acc [0.93112355 0.89287109 0.85815255 0.80213037] 

2019-02-23T23:22:31.268239: step 2840, loss 0.636168, acc [0.91826172 0.87490234 0.82226562 0.76962891]
2019-02-23T23:22:33.267119: step 2880, loss 0.644285, acc [0.92070312 0.87363281 0.8234375  0.76982422]
2019-02-23T23:22:35.261537: step 2920, loss 0.628392, acc [0.91884766 0.87246094 0.82783203 0.77119141]
2019-02-23T23:22:37.252478: step 2960, loss 0.619

2019-02-23T23:25:12.844877: step 5200, loss 0.359414, acc [0.94832264 0.92444614 0.9027921  0.85308693] 

2019-02-23T23:25:14.969292: step 5240, loss 0.297976, acc [0.95419922 0.92685547 0.90732422 0.85527344]
2019-02-23T23:25:16.976847: step 5280, loss 0.317823, acc [0.95078125 0.92246094 0.90351563 0.84785156]
2019-02-23T23:25:18.998008: step 5320, loss 0.313422, acc [0.95214844 0.92480469 0.90410156 0.85400391]
2019-02-23T23:25:21.020229: step 5360, loss 0.309869, acc [0.9546875  0.92685547 0.90556641 0.85556641]
2019-02-23T23:25:23.012134: step 5400, loss 0.328198, acc [0.94794922 0.92578125 0.89824219 0.84873047]

Evaluation:
2019-02-23T23:25:26.458340: step 5400, loss 0.3553, acc [0.94902342 0.92490665 0.90362302 0.85371763] 

2019-02-23T23:25:28.553941: step 5440, loss 0.324988, acc [0.94726562 0.92392578 0.90292969 0.85107422]
2019-02-23T23:25:30.604403: step 5480, loss 0.328324, acc [0.94863281 0.92011719 0.90058594 0.84560547]
2019-02-23T23:25:32.634036: step 5520, loss 0.328

2019-02-23T23:28:11.531050: step 7800, loss 0.287837, acc [0.95833375 0.94206569 0.92605792 0.88266976] 

2019-02-23T23:28:13.800214: step 7840, loss 0.176457, acc [0.96725162 0.95399404 0.94486663 0.9005376 ]
2019-02-23T23:28:15.841253: step 7880, loss 0.152352, acc [0.96728516 0.95507812 0.95048828 0.90615234]
2019-02-23T23:28:17.886298: step 7920, loss 0.145707, acc [0.96962891 0.95820313 0.95429688 0.91132813]
2019-02-23T23:28:19.923333: step 7960, loss 0.151341, acc [0.96933594 0.95742187 0.9515625  0.90839844]
2019-02-23T23:28:21.973797: step 8000, loss 0.152499, acc [0.96757812 0.95761719 0.95078125 0.90664062]

Evaluation:
2019-02-23T23:28:25.404868: step 8000, loss 0.299989, acc [0.95941495 0.94195557 0.92767972 0.88510246] 

2019-02-23T23:28:27.530227: step 8040, loss 0.159248, acc [0.96738281 0.95722656 0.94746094 0.90322266]
2019-02-23T23:28:29.578212: step 8080, loss 0.160425, acc [0.96904297 0.95429688 0.94746094 0.90185547]
2019-02-23T23:28:31.602883: step 8120, loss 0.1

2019-02-23T23:31:10.524188: step 10400, loss 0.281217, acc [0.96292885 0.94802231 0.93650952 0.89852737] 

2019-02-23T23:31:12.666857: step 10440, loss 0.153128, acc [0.96582031 0.95742187 0.95009766 0.90644531]
2019-02-23T23:31:14.740634: step 10480, loss 0.151734, acc [0.97148437 0.96015625 0.94931641 0.90908203]
2019-02-23T23:31:16.798535: step 10520, loss 0.167277, acc [0.96728516 0.95361328 0.94560547 0.90380859]
2019-02-23T23:31:18.844534: step 10560, loss 0.158901, acc [0.96962891 0.95429688 0.94833984 0.90488281]
2019-02-23T23:31:20.865239: step 10600, loss 0.173289, acc [0.96650391 0.95126953 0.94443359 0.89931641]

Evaluation:
2019-02-23T23:31:24.317397: step 10600, loss 0.27919, acc [0.96412017 0.94885323 0.93771086 0.89952848] 

2019-02-23T23:31:26.480949: step 10640, loss 0.168931, acc [0.96660156 0.95634766 0.94335938 0.90263672]
2019-02-23T23:31:28.528255: step 10680, loss 0.173349, acc [0.96533203 0.95566406 0.94443359 0.90107422]
2019-02-23T23:31:30.561855: step 10720,

...

2019-02-24T00:09:10.070869: step 43400, loss 0.399437, acc [0.97785542 0.96721361 0.95735266 0.93989328] 

2019-02-24T00:09:12.203173: step 43440, loss 0.0374671, acc [0.99238281 0.98740234 0.98642578 0.97148437]
2019-02-24T00:09:14.269508: step 43480, loss 0.0357178, acc [0.990625   0.99003906 0.98798828 0.97333984]
2019-02-24T00:09:16.334855: step 43520, loss 0.0416303, acc [0.99033203 0.98867187 0.98720703 0.97177734]
2019-02-24T00:09:18.410115: step 43560, loss 0.0384045, acc [0.99121094 0.98974609 0.98789063 0.97275391]
2019-02-24T00:09:20.471049: step 43600, loss 0.0343775, acc [0.99267578 0.98867187 0.98779297 0.97373047]

Evaluation:
2019-02-24T00:09:23.942497: step 43600, loss 0.385381, acc [0.97840603 0.96787434 0.95747279 0.9408143 ] 

2019-02-24T00:09:26.099601: step 43640, loss 0.0400452, acc [0.99082031 0.9875     0.98583984 0.96923828]
2019-02-24T00:09:28.180817: step 43680, loss 0.0403739, acc [0.99003906 0.98720703 0.98691406 0.96992188]
2019-02-24T00:09:30.371646: ste

2019-02-24T00:12:06.289884: step 45960, loss 0.0253225, acc [0.9921875  0.99335938 0.99082031 0.97919922]
2019-02-24T00:12:08.342331: step 46000, loss 0.0264121, acc [0.99287109 0.99208984 0.99042969 0.97744141]

Evaluation:
2019-02-24T00:12:11.870874: step 46000, loss 0.429293, acc [0.97788545 0.96782429 0.95765299 0.9411747 ] 

2019-02-24T00:12:14.024505: step 46040, loss 0.0337062, acc [0.99414062 0.98925781 0.98994141 0.97666016]
2019-02-24T00:12:16.119610: step 46080, loss 0.0321546, acc [0.99248047 0.98964844 0.99013672 0.97636719]
2019-02-24T00:12:18.177017: step 46120, loss 0.0334172, acc [0.9921875  0.9921875  0.98896484 0.97626953]
2019-02-24T00:12:20.259224: step 46160, loss 0.0378142, acc [0.99248047 0.98818359 0.98671875 0.97128906]
2019-02-24T00:12:22.321591: step 46200, loss 0.0273571, acc [0.99238281 0.99248047 0.99042969 0.97763672]

Evaluation:
2019-02-24T00:12:25.835320: step 46200, loss 0.416709, acc [0.97789546 0.96795443 0.9570223  0.94014356] 

2019-02-24T00:12:2

2019-02-24T00:15:05.073967: step 48480, loss 0.0202245, acc [0.99248047 0.99296875 0.99296875 0.98085937]
2019-02-24T00:15:07.151167: step 48520, loss 0.0220251, acc [0.99306641 0.99404297 0.99306641 0.98251953]
2019-02-24T00:15:09.236846: step 48560, loss 0.0264577, acc [0.99228516 0.99228516 0.99130859 0.97998047]
2019-02-24T00:15:11.352285: step 48600, loss 0.0238325, acc [0.99482422 0.99228516 0.99189453 0.98125   ]

Evaluation:
2019-02-24T00:15:15.091166: step 48600, loss 0.418053, acc [0.97795553 0.96869525 0.9584739  0.94204567] 

2019-02-24T00:15:17.271546: step 48640, loss 0.027061, acc [0.99365234 0.99228516 0.99199219 0.98076172]
2019-02-24T00:15:19.368137: step 48680, loss 0.0287043, acc [0.99384766 0.99365234 0.99072266 0.98154297]
2019-02-24T00:15:21.475146: step 48720, loss 0.0233847, acc [0.99384766 0.99033203 0.99199219 0.97929687]
2019-02-24T00:15:23.561378: step 48760, loss 0.0213869, acc [0.99365234 0.99355469 0.99277344 0.98183594]
2019-02-24T00:15:25.657912: step 

2019-02-24T00:18:03.748046: step 51000, loss 0.425372, acc [0.97893662 0.9685551  0.95818358 0.94231597] 

2019-02-24T00:18:05.914026: step 51040, loss 0.02726, acc [0.99160156 0.990625   0.99003906 0.97519531]
2019-02-24T00:18:07.965979: step 51080, loss 0.0269527, acc [0.99365234 0.99208984 0.98974609 0.97792969]
2019-02-24T00:18:10.032316: step 51120, loss 0.021722, acc [0.99384766 0.99179688 0.99316406 0.98105469]
2019-02-24T00:18:12.163626: step 51160, loss 0.0282983, acc [0.99335938 0.99160156 0.99121094 0.97890625]
2019-02-24T00:18:14.235417: step 51200, loss 0.028322, acc [0.99277344 0.99130859 0.99160156 0.97929687]

Evaluation:
2019-02-24T00:18:17.775863: step 51200, loss 0.42697, acc [0.97856621 0.96857512 0.9588243  0.94254622] 

2019-02-24T00:18:19.944962: step 51240, loss 0.0297882, acc [0.99296875 0.99199219 0.98974609 0.97734375]
2019-02-24T00:18:22.025186: step 51280, loss 0.0310241, acc [0.99140625 0.99179688 0.99072266 0.97753906]
2019-02-24T00:18:24.102931: step 513

KeyboardInterrupt: 

In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape